# Computer vision problems

Vorige week heb je geleerd hoe je gebruik kan maken van convolutionele lagen om op een efficiente manier classificatie problemen uit te voeren op beelden.
Echter zijn er naast classificatie problemen ook andere zaken die je kan doen in het domein van computer visie.

De meest voorkomende hiervan zijn:
* Object Localization
* Object Detection 
* Image Segmentation

Modellen voor deze oplossingen kunnen zelf gemaakt worden (een oefening hierop gaan we uitvoeren bij image segmentation).
Echter zijn ze vaak heel complex waardoor het vaak eenvoudiger en sneller is om bestaande oplossingen/modellen hiervoor te gebruiken.
Gelukkig zijn er hiervoor een hele reeks te vinden op tensorflow hub.

## Object Localization

Hierbij wordt de regio aangeduid van elke klasse waarin een object gezien wordt.
Dit houdt echter in dat er slechts maar 1 object van elke klasse kan gedetecteerd worden.

Dit kan eenvoudig gedaan worden door het classificatieprobleem aan te passen waarbij je ipv 1 neuron te gebruiken er 4 gebruikt (of 5).
Deze 4 of 5 neurons kunnen dan bijvoorbeeld staan voor:
* xmin van de bounding box
* ymin van de bounding box
* width van de bouding box
* height van de bouding box
* optionele zekerheid van de classe/bounding box

## Object Detection

Object detectie is een uitbreiding van object localization waarbij geprobeerd wordt alle objecten in een figuur te detecteren en classificeren.
Er zijn doorheen de jaren een hele reeks neurale netwerken getrained voor verschillende inputs/classes/snelheden/... 
Een overzicht hiervan kan je vinden op [deze link](https://tfhub.dev/tensorflow/collections/object_detection/1)

**Oefening**

Voer de volgende stappen uit:
* Kies een model uit de lijst van tensorflow hub
* Bekijk de colab notebook en bestudeer de code uit de verschillende cellen
* Haal er de minimum code uit om het neuraal netwerk te downloaden en te gebruiken
* Kies een figuur op het internet en voer er object detectie op uit met je zelfgekozen neuraal netwerk
* Maak een plot van de input figuur met de gedetecteerde objecten erop getekend. Let op bij het weergeven van de namen van de classen. Dit lukt niet altijd zoals in de notebook op tensorflow. (Tip: kijk naar de notebook op tensorflow en de democode uit lesweek 1)

In [ ]:
labels = []

with open('ImageNetLabels.txt') as f:
    lines = f.readlines()

    for line in lines:
        labels.append(line.replace("\n", ""))

labels = ['person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','fire hydrant','stop sign','parking meter','bench','bird','cat','dog','horse','sheep','cow','elephant','bear','zebra','giraffe','backpack','umbrella','handbag','tie','suitcase','frisbee','skis','snowboard','sports ball','kite','baseball bat','baseball glove','skateboard','surfboard','tennis racket','bottle','wine glass','cup','fork','knife','spoon','bowl','banana','apple','sandwich','orange','broccoli','carrot','hot dog','pizza','donut','cake','chair','couch','potted plant','bed','dining table','toilet','tv','laptop','mouse','remote','keyboard','cell phone','microwave','oven','toaster','sink','refrigerator','book','clock','vase','scissors','teddy bear','hair drier','toothbrush']

In [ ]:
# load an object detection model from tensorflow hub
# load the same image as in the previous example
# make the prediction for the image
# show image and predicted objects

# Image segmentation

Een uitbreiding op object detectie is dat we gaan kijken voor elke pixel of groep pixels tot welke klasse het behoort.
Ook hiervoor bestaan er een reeks bestaande netwerken op tensorflow hub, zie [deze link](https://tfhub.dev/s?module-type=image-segmentation).

Let op dat het Mask R-CNN model hier niet tussen staat omdat het technisch gezien object detectie doet maar de resultaten kunnen ook gebruikt worden voor image segmentation.
Bekijk de notebook voor object detection met mask r-cnn en voer image segmentation uit met dezelfde figuur.

Let op dat ik bij het voorbereiden door een versieverschil een fout kreeg met de standaard reframe_box_masks_to_image_masks functie.
Indien je hier ook een fout mee krijgt kan je de onderstaande functie gebruiken. (Aanpassing staat in commentaar)

In [ ]:
# copy from package and fixed?
def reframe_box_masks_to_image_masks(box_masks, boxes, image_height,
                                     image_width):
  """Transforms the box masks back to full image masks.

  Embeds masks in bounding boxes of larger masks whose shapes correspond to
  image shape.

  Args:
    box_masks: A tf.float32 tensor of size [num_masks, mask_height, mask_width].
    boxes: A tf.float32 tensor of size [num_masks, 4] containing the box
           corners. Row i contains [ymin, xmin, ymax, xmax] of the box
           corresponding to mask i. Note that the box corners are in
           normalized coordinates.
    image_height: Image height. The output mask will have the same height as
                  the image height.
    image_width: Image width. The output mask will have the same width as the
                 image width.

  Returns:
    A tf.float32 tensor of size [num_masks, image_height, image_width].
  """
  # TODO(rathodv): Make this a public function.
  def reframe_box_masks_to_image_masks_default():
    """The default function when there are more than 0 box masks."""
    def transform_boxes_relative_to_boxes(boxes, reference_boxes):
      boxes = tf.reshape(boxes, [-1, 2, 2])
      min_corner = tf.expand_dims(reference_boxes[:, 0:2], 1)
      max_corner = tf.expand_dims(reference_boxes[:, 2:4], 1)
      transformed_boxes = (boxes - min_corner) / (max_corner - min_corner)
      return tf.reshape(transformed_boxes, [-1, 4])

    box_masks_expanded = tf.expand_dims(box_masks, axis=3)
    num_boxes = tf.shape(box_masks_expanded)[0]
    unit_boxes = tf.concat(
        [tf.zeros([num_boxes, 2]), tf.ones([num_boxes, 2])], axis=1)
    reverse_boxes = transform_boxes_relative_to_boxes(unit_boxes, boxes)
    return tf.image.crop_and_resize(
        image=box_masks_expanded,
        boxes=reverse_boxes,
        box_indices=tf.range(num_boxes),    # dit heb ik aangepast (stond oorspronkelijk box_ind)
        crop_size=[image_height, image_width],
        extrapolation_value=0.0)
  image_masks = tf.cond(
      tf.shape(box_masks)[0] > 0,
      reframe_box_masks_to_image_masks_default,
      lambda: tf.zeros([0, image_height, image_width, 1], dtype=tf.float32))
  return tf.squeeze(image_masks, axis=3)


In [ ]:
# load image segmentation model from tensorflow hub
# load the same image as in the previous example
# make the prediction for the image

In [ ]:
# show original image and its mask

Natuurlijk is het ook mogelijk om dit soort netwerken zelf te maken. 
Als experiment gaan we op beelden de soort en ras van huisdieren gaan bepalen.
Hiervoor gaan we gebruik maken van [deze dataset](https://www.kaggle.com/datasets/devdgohil/the-oxfordiiit-pet-dataset).

Download in de cell hieronder de dataset en bekijk de files die gedownload zijn.

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/devdgohil/the-oxfordiiit-pet-dataset")

Er zijn twee folders gedownload. 
De folder images bevat alle figuren in de dataset waarbij de naam van de file begint met de klasse waartoe het behoort.
De folder annotations is iets complexer en bevat segmentation masks voor elke figuur in de trimaps folder en extra metadata voor elke figuur in de xmls file.
Deze metadata bevat bijvoorbeeld informatie over de bounding boxes van de dieren in de figuren.
In dit deel gaan we enkel werken met de informatie uit de trimaps folder om het model te trainen.

In [ ]:
input_dir = "the-oxfordiiit-pet-dataset/images/images"
target_dir = "the-oxfordiiit-pet-dataset/annotations/annotations/trimaps"

import os
input_img_paths = sorted([os.path.join(input_dir,fname) for fname in os.listdir(input_dir) if fname.endswith(".jpg")])
target_paths = sorted([os.path.join(target_dir,fname) for fname in os.listdir(target_dir) if fname.endswith(".png") and not fname.startswith(".")])

print("Num inputs: ", len(input_img_paths))
print("Num targets: ", len(target_paths))

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

In [ ]:
# show some input images (2 rows of 8 images)

In [ ]:
# show some target images (2 rows of 8 images)

In [ ]:
# the figures have different sizes => make everything 200x200

# set some images aside for validation

# maak een Data Sequence Generator voor train en validatie data

In [ ]:
# build and make model

# print summary - how many parameters do you have?

# compile the model and pick an optimizer and loss function

In [ ]:
# add call back for saving best model

# train the model

In [ ]:
# show a random target image

In [ ]:
# load best model (saved by checkpoints) instead of model resulting from final fit

# display mask over the random target image